In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

import urllib.request
import matplotlib.dates as mdates
import datetime as Date
from geopy.geocoders import Nominatim
pd.set_option('display.max_columns',None)










In [ ]:
# Load and preprocess the dataset

url = "https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M_MAX,T2M_MIN,RH2M,PRECTOTCORR,PS,WS50M_MAX,WS50M_MIN&community=RE&longitude=71.4000&latitude=19.2000&start=20050101&end=20221231&format=CSV"
# Assuming the dataset is in a CSV file with columns: date, param1, param2, ..., param7

urllib.request.urlretrieve(url,'climate.csv')
df = pd.read_csv('climate.csv',skiprows = 15)

df['YEAR'] = df.YEAR.astype(str)
df['MO'] = df.MO.astype(str)
df['DY'] = df.DY.astype(str)

df['date'] = df['YEAR'].str.cat(df['MO'],sep = '/')
df['DATE'] = df['date'].str.cat(df['DY'],sep = '/')
df.drop(columns = ['YEAR','MO','DY','date','DATE'],axis=1,inplace=True)

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.values)

# Convert the data to PyTorch tensors
data = torch.from_numpy(df_scaled).float()

In [ ]:
data.shape

torch.Size([6574, 7])

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn1 = nn.RNN(input_size, hidden_size, batch_first=True)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.5)
        
        self.rnn2 = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.5)
        
        self.rnn3 = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.5)
        
        self.rnn4 = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.bn4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(0.5)
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden1 = self.init_hidden(batch_size)
        hidden2 = self.init_hidden(batch_size)
        hidden3 = self.init_hidden(batch_size)
        hidden4 = self.init_hidden(batch_size)

        out, hidden1 = self.rnn1(x, hidden1)
        # out = self.dropout1(self.bn1(out))
        out, hidden2 = self.rnn2(out, hidden2)
        # out = self.dropout2(self.bn2(out))
        out, hidden3 = self.rnn3(out, hidden3)
        # out = self.dropout3(self.bn3(out))
        out, hidden4 = self.rnn4(out, hidden4)
        # out = self.dropout4(self.bn4(out))
        
        out = self.fc(out[:, -1:, :]) # select the output for the last 1 days
        return out

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

In [ ]:
# Initialize hyperparameters
input_size = 7  # 7 weather parameters
hidden_size = 64   # specify your desired hidden size
num_layers = 2     # specify your desired number of layers
output_size = 7   # 7 weather parameters x 7 days
seq_length = 15  # Number of previous days to consider as input
target_length = 1 # Number of future days to consider as output
batch_size = 32  # Batch size for training
learning_rate = 0.001  # Learning rate for optimizer
num_epochs = 50  # Number of epochs for training



In [ ]:
# Create input sequences and targets for predicting the next 7 days
X = []
y = []
for i in range(len(data) - seq_length - target_length):
    seq = data[i:i + seq_length, :]
    target = data[i + seq_length:i + seq_length + target_length, :]
    X.append(seq)
    y.append(target)
X = torch.stack(X)
y = torch.stack(y)

# Calculate sizes for train, validation, and test sets
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))
test_size = len(X) - train_size - val_size

# Split the dataset into train, validation, and test sets
train_X, val_X, test_X = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
train_y, val_y, test_y = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

print(train_X.shape)
print(train_y.shape)
print(val_X.shape)
print(val_y.shape)
print(test_X.shape)
print(test_y.shape)

# Create DataLoader for training set
train_dataset = torch.utils.data.TensorDataset(train_X, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create DataLoader for validation set
val_dataset = torch.utils.data.TensorDataset(val_X, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Create DataLoader for test set
test_dataset = torch.utils.data.TensorDataset(test_X, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


torch.Size([4590, 15, 7])
torch.Size([4590, 1, 7])
torch.Size([983, 15, 7])
torch.Size([983, 1, 7])
torch.Size([985, 15, 7])
torch.Size([985, 1, 7])


In [ ]:
import matplotlib.pyplot as plt

# Initialize the model, loss function, and optimizer
# Create an instance of the hybrid model
model = RNNModel(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean squared error loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

train_losses = []  # List to store training losses for each epoch
val_losses = []  # List to store validation losses for each epoch
best_loss = float('inf')  # Set initial best loss to positive infinity
best_model_state_dict = None  # Variable to store the state_dict of the best model

for epoch in range(num_epochs):
    epoch_train_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    epoch_train_loss /= len(train_loader)
    train_losses.append(epoch_train_loss)

    # Evaluate the model on the validation set
    model.eval()
    epoch_val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            epoch_val_loss += criterion(outputs, targets).item()
    epoch_val_loss /= len(val_loader)
    val_losses.append(epoch_val_loss)

    # Update the best model if the current model has a lower validation loss
    if epoch_val_loss < best_loss:
        best_loss = epoch_val_loss
        best_model_state_dict = model.state_dict()
        torch.save(best_model_state_dict, 'best_model.pth')  # Save the best model to a file

    model.train()

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}')

# Plot the training and validation loss graphs
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(f'Training complete. Best model saved with validation loss: {best_loss:.4f}')


In [ ]:
# Evaluate the model on the test data
# Load the saved state_dict of the best model
best_model_state_dict = torch.load('best_model.pth')

# Set the model's state_dict to the loaded state_dict
model.load_state_dict(best_model_state_dict)

model.eval()
with torch.no_grad():
    test_outputs = model(test_X)
    test_loss = criterion(test_outputs, test_y)
print('Test Loss: {:.4f}'.format(test_loss.item()))

In [ ]:
# Make predictions on the test data
model.eval()
with torch.no_grad():
    predicted = model(test_X)

# # # Denormalize the predicted values
# denormalized_tensor = scaler.inverse_transform(predicted.view(-1, 7)).view(580, 7, 7)
# denormalized_tensor_y = scaler.inverse_transform(test_y.view(-1, 7)).view(580, 7, 7)
# predicted.shape




    

In [ ]:
denormalized_tensor = torch.empty_like(predicted)
denormalized_tensor_y = torch.empty_like(test_y)
for i in range(predicted.shape[0]):
    for j in  range(0,1):
      num = predicted[i][j].numpy()
      nump = np.array(num)
      numpp = nump.reshape(1,7)
      denorm = scaler.inverse_transform(numpp)
      denormm = torch.from_numpy(denorm)
      denormalized_tensor[i][j] = denormm
for i in range(test_y.shape[0]):
    for j in  range(0,1):
      num = test_y[i][j].numpy()
      nump = np.array(num)
      numpp = nump.reshape(1,7)
      denorm = scaler.inverse_transform(numpp)
      denormm = torch.from_numpy(denorm)
      denormalized_tensor_y[i][j] = denormm
for i in range(0,10):
  print(denormalized_tensor[i])
  print("/////")
  print(denormalized_tensor_y[i])
  print("//////////////////////////////////////////////////////////////////////////")

In [ ]:
for i in range(0,10):
  print(train_X[i])
  print("/////")
  print(train_y[i])
  print("//////////////////////////////////////////////////////////////////////////")

In [ ]:
model = RNNModel(input_size, hidden_size, output_size)
from torchsummary import summary
input_e = (15, 7)
input_tensor = torch.randn(input_size)
summary(model, input_size=input_e)

In [ ]:

import time

# Set your model to evaluation mode
model.eval()

# Define the device to use (e.g., "cuda" for GPU, "cpu" for CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device
model.to(device)

# Initialize variables for total inference time and total number of batches
total_time = 0.0
total_batches = 0

# Iterate through the test loader
for batch_idx, (input, target) in enumerate(test_loader):
    # Move the batch of test data to the appropriate device
    input = input.to(device)

    # Measure the start time of inference
    start_time = time.time()

    # Disable gradient computation during inference
    with torch.no_grad():
        # Pass the batch of test data through the model
        output = model(input)

    # Measure the end time of inference
    end_time = time.time()

    # Compute the inference time for the batch
    inference_time = end_time - start_time

    # Accumulate the inference time and update the total number of batches
    total_time += inference_time
    total_batches += 1

# Calculate the average inference time
avg_inference_time = total_time / total_batches

# Print the average inference time in seconds
print("Average Inference Time: {:.4f} seconds".format(avg_inference_time))


In [ ]:
total_params = sum(p.numel() for p in model.fc.parameters() if p.requires_grad)
print("Total trainable parameters in GRU layer:", total_params)